In [2]:
import pandas as pd
from pandas import json_normalize
import requests
import json
import snowflake.connector

from dotenv import load_dotenv
import os
import http.client
import urllib.parse


from googletrans import Translator
import string
import re

from sqlalchemy import create_engine

import asyncio

/home/nxie/code/nikki099/de_linkedin_etl_project/.venv/lib/python3.11/site-packages/snowflake/connector/options.py:104: UserWarning: You have an incompatible version of 'pyarrow' installed (20.0.0), please install a version that adheres to: 'pyarrow<19.0.0; extra == "pandas"'
  warn_incompatible_dep(


In [2]:
load_dotenv()
rapidapi_key = os.getenv('RAPIDAPI_KEY')
rapidapi_host = "linkedin-job-search-api.p.rapidapi.com"
snowflake_user = os.getenv('SNOWFLAKE_USER')
account = os.getenv('SNOWFLAKE_ACCOUNT')
snowflake_password = os.getenv('SNOWFLAKE_PASSWORD')

In [3]:
def extract_linkedin_job_data():

    headers = {
        'x-rapidapi-key': rapidapi_key,
        'x-rapidapi-host': rapidapi_host
    }

    location = "Australia"
    limit = 100
    offset = 0
    titles = ["Data Engineer" , "Data Scientist", "Data Analyst"]

    df_daily_all = pd.DataFrame()

    for title_filter in titles:

        #URL code the title filter and location filter
        title_encoded=urllib.parse.quote(title_filter)
        location_encoded=urllib.parse.quote(location)

        #API endpoint 7day data query
        #base_url = f"/active-jb-7d?limit={limit}&offset={offset}&title_filter={title_encoded}&location_filter={location_encoded}"

        #API endpoint last 24 hour data query
        base_url = f"/active-jb-24h?limit={limit}&offset={offset}&title_filter={title_encoded}&location_filter={location_encoded}"
        url =  f"https://{rapidapi_host}{base_url}"


        response = requests.get(url, headers=headers)

        if response.status_code == 200:
            data = response.json()
            # Convert the JSON data to a DataFrame
            df_daily = json_normalize(data)
            df_daily['job_category'] = title_filter
            print(title_filter, df_daily.shape)
            df_daily_all = pd.concat([df_daily_all, df_daily], ignore_index=True)


        else:
            print(f"Error: {response.status_code}")

    return df_daily_all

df_daily_all = extract_linkedin_job_data()
# df_daily_all

Data Engineer (23, 51)
Data Scientist (23, 45)
Data Analyst (14, 51)


In [4]:
# df_daily_all = pd.read_csv('linkedin_jobs_daily.csv')
df_daily_all.head()

,id,date_posted,date_created,title,organization,organization_url,date_validthrough,locations_raw,location_type,location_requirements_raw,...,seniority,directapply,linkedin_org_slug,salary_raw.@type,salary_raw.currency,salary_raw.value.@type,salary_raw.value.minValue,salary_raw.value.maxValue,salary_raw.value.unitText,job_category
0,1657217375,2025-05-14T05:01:19,2025-05-14T05:33:27.078778,Senior Engineer - Data Engineering,Slalom,https://www.linkedin.com/company/slalom-consul...,2025-06-13T05:01:19,"[{'@type': 'Place', 'address': {'@type': 'Post...",None,None,...,Mid-Senior level,False,slalom-consulting,NaN,NaN,NaN,NaN,NaN,NaN,Data Engineer
1,1657217275,2025-05-14T05:01:19,2025-05-14T05:33:22.122477,Senior Engineer - Data Engineering,Slalom,https://www.linkedin.com/company/slalom-consul...,2025-06-13T05:01:19,"[{'@type': 'Place', 'address': {'@type': 'Post...",None,None,...,Mid-Senior level,False,slalom-consulting,NaN,NaN,NaN,NaN,NaN,NaN,Data Engineer
2,1656925568,2025-05-14T03:20:29,2025-05-14T03:32:52.435808,Data Engineer with Databricks,BURGEON IT SERVICES,https://www.linkedin.com/company/burgeon-it-se...,2025-06-13T03:20:29,"[{'@type': 'Place', 'address': {'@type': 'Post...",None,None,...,Mid-Senior level,True,burgeon-it-services-llc,NaN,NaN,NaN,NaN,NaN,NaN,Data Engineer
3,1656683181,2025-05-14T01:55:49,2025-05-14T02:04:37.175479,Data Engineer - DataStage | REDSHIFT | SQL | P...,Resource Corner,https://www.linkedin.com/company/resource-corn...,2025-06-13T01:55:49,"[{'@type': 'Place', 'address': {'@type': 'Post...",None,None,...,Entry level,True,resource-cornerit,NaN,NaN,NaN,NaN,NaN,NaN,Data Engineer
4,1656683135,2025-05-14T01:54:34,2025-05-14T02:04:36.193784,Senior Data Engineer,GROW Inc,https://www.linkedin.com/company/grow-inc,2025-06-13T01:53:47,"[{'@type': 'Place', 'address': {'@type': 'Post...",None,None,...,Mid-Senior level,True,grow-inc,NaN,NaN,NaN,NaN,NaN,NaN,Data Engineer


In [5]:
def get_clean_data_jobs(df_daily_all):
     pattern=re.compile(r'\bData Engineer\b|\bData Scientist\b|\bData Analyst\b' , re.IGNORECASE)
     df_daily_all = df_daily_all[df_daily_all['title'].str.contains(pattern)]
     return df_daily_all

df_daily_all = get_clean_data_jobs(df_daily_all)
df_daily_all.shape

(53, 51)

In [6]:
df_daily_all.to_csv('linkedin_jobs_daily.csv', index=False)

In [7]:
df_daily_all = pd.read_csv('linkedin_jobs_daily.csv')
df_daily_all.shape

(53, 51)

In [8]:
def update_columns(df_daily_all):
    df_daily_all.columns = df_daily_all.columns.str.upper()
    df_daily_all = df_daily_all[['ID', 'DATE_POSTED', 'DATE_CREATED', 'TITLE', 'JOB_CATEGORY',
       'ORGANIZATION', 'ORGANIZATION_URL', 'DATE_VALIDTHROUGH', 'LOCATIONS_RAW',
       'LOCATION_TYPE', 'LOCATION_REQUIREMENTS_RAW', 'EMPLOYMENT_TYPE', 'URL',
       'SOURCE_TYPE', 'SOURCE', 'SOURCE_DOMAIN', 'ORGANIZATION_LOGO',
       'CITIES_DERIVED', 'REGIONS_DERIVED', 'COUNTRIES_DERIVED',
       'LOCATIONS_DERIVED', 'TIMEZONES_DERIVED', 'LATS_DERIVED',
       'LNGS_DERIVED', 'REMOTE_DERIVED', 'RECRUITER_NAME', 'RECRUITER_TITLE',
       'RECRUITER_URL', 'LINKEDIN_ORG_EMPLOYEES', 'LINKEDIN_ORG_URL',
       'LINKEDIN_ORG_SIZE', 'LINKEDIN_ORG_SLOGAN', 'LINKEDIN_ORG_INDUSTRY',
       'LINKEDIN_ORG_FOLLOWERS', 'LINKEDIN_ORG_HEADQUARTERS',
       'LINKEDIN_ORG_TYPE', 'LINKEDIN_ORG_FOUNDEDDATE',
       'LINKEDIN_ORG_SPECIALTIES', 'LINKEDIN_ORG_LOCATIONS',
       'LINKEDIN_ORG_DESCRIPTION', 'LINKEDIN_ORG_RECRUITMENT_AGENCY_DERIVED',
       'SENIORITY', 'DIRECTAPPLY', 'LINKEDIN_ORG_SLUG']]
    return df_daily_all

df_daily_all = update_columns(df_daily_all)
df_daily_all.head(2)

,ID,DATE_POSTED,DATE_CREATED,TITLE,JOB_CATEGORY,ORGANIZATION,ORGANIZATION_URL,DATE_VALIDTHROUGH,LOCATIONS_RAW,LOCATION_TYPE,...,LINKEDIN_ORG_HEADQUARTERS,LINKEDIN_ORG_TYPE,LINKEDIN_ORG_FOUNDEDDATE,LINKEDIN_ORG_SPECIALTIES,LINKEDIN_ORG_LOCATIONS,LINKEDIN_ORG_DESCRIPTION,LINKEDIN_ORG_RECRUITMENT_AGENCY_DERIVED,SENIORITY,DIRECTAPPLY,LINKEDIN_ORG_SLUG
0,1656925568,2025-05-14T03:20:29,2025-05-14T03:32:52.435808,Data Engineer with Databricks,Data Engineer,BURGEON IT SERVICES,https://www.linkedin.com/company/burgeon-it-se...,2025-06-13T03:20:29,"[{'@type': 'Place', 'address': {'@type': 'Post...",NaN,...,"Claymont, Delaware",Privately Held,2010.0,"['IT Consulting', 'Contract Staffing', 'Perman...","['619,, New York Avenue, Claymont, Delaware 19...",Burgeon is a provider of technical and IT staf...,True,Mid-Senior level,True,burgeon-it-services-llc
1,1656683181,2025-05-14T01:55:49,2025-05-14T02:04:37.175479,Data Engineer - DataStage | REDSHIFT | SQL | P...,Data Engineer,Resource Corner,https://www.linkedin.com/company/resource-corn...,2025-06-13T01:55:49,"[{'@type': 'Place', 'address': {'@type': 'Post...",NaN,...,"Norwest, New South Wales",Privately Held,2006.0,[''],"['4 Columbia Court, Norwest Business Park, Lev...",Resource Corner is a Leading IT Consulting and...,False,Entry level,True,resource-cornerit


### Process the df_daily_all data 


In [9]:
def extract_job_date(date_created):
    #extract the job_date from job_created
    df_daily_all['job_date'] = pd.to_datetime(df_daily_all['DATE_CREATED']).dt.date
    return df_daily_all['job_date']

#Apply the function to the DataFrame
df_daily_all['job_date'] = extract_job_date(df_daily_all['DATE_CREATED'])
df_daily_all['job_date'][:5]


0    2025-05-14
1    2025-05-14
2    2025-05-14
3    2025-05-14
4    2025-05-14
Name: job_date, dtype: object

In [10]:
#Extrac the job city and state from the LOCATIONS_RAW field

def extract_city(list):
    """
    Extracts the city from the given text using regex.
    """
    # Regex pattern to match the city in the LOCATIONS_RAW field
    text = str(list)
    city_pattern = r"'addressLocality':\s*'(.*)',\s'addressRegion':"
    match = re.search(city_pattern, text)
    if match:
        city = match.group(1)
        if 'sidney' in city.lower() or 'sídney' in city.lower() or '悉尼' in city.lower(): #fix wrong city name
            return "Sydney"
        return city
    else:
        return None



def extract_state(list):
    """
    Extracts the state from the given text using regex.
    """
    # Regex pattern to match the region/state in the LOCATIONS_RAW field
    text = str(list)
    state_pattern = r"'addressRegion':\s*(.*)',\s'streetAddress'"
    match = re.search(state_pattern, text)
    if match:
        state = match.group(1)
        state= state.replace("'", "").strip()
        return state
    else:
        return None


#Extract city and state from Locations Raw
df_daily_all['city'] = df_daily_all['LOCATIONS_RAW'].apply(extract_city)


df_daily_all['state'] = df_daily_all['LOCATIONS_RAW'].apply(extract_state)



df_daily_all.head()

,ID,DATE_POSTED,DATE_CREATED,TITLE,JOB_CATEGORY,ORGANIZATION,ORGANIZATION_URL,DATE_VALIDTHROUGH,LOCATIONS_RAW,LOCATION_TYPE,...,LINKEDIN_ORG_SPECIALTIES,LINKEDIN_ORG_LOCATIONS,LINKEDIN_ORG_DESCRIPTION,LINKEDIN_ORG_RECRUITMENT_AGENCY_DERIVED,SENIORITY,DIRECTAPPLY,LINKEDIN_ORG_SLUG,job_date,city,state
0,1656925568,2025-05-14T03:20:29,2025-05-14T03:32:52.435808,Data Engineer with Databricks,Data Engineer,BURGEON IT SERVICES,https://www.linkedin.com/company/burgeon-it-se...,2025-06-13T03:20:29,"[{'@type': 'Place', 'address': {'@type': 'Post...",NaN,...,"['IT Consulting', 'Contract Staffing', 'Perman...","['619,, New York Avenue, Claymont, Delaware 19...",Burgeon is a provider of technical and IT staf...,True,Mid-Senior level,True,burgeon-it-services-llc,2025-05-14,Melbourne,VIC
1,1656683181,2025-05-14T01:55:49,2025-05-14T02:04:37.175479,Data Engineer - DataStage | REDSHIFT | SQL | P...,Data Engineer,Resource Corner,https://www.linkedin.com/company/resource-corn...,2025-06-13T01:55:49,"[{'@type': 'Place', 'address': {'@type': 'Post...",NaN,...,[''],"['4 Columbia Court, Norwest Business Park, Lev...",Resource Corner is a Leading IT Consulting and...,False,Entry level,True,resource-cornerit,2025-05-14,Brisbane,QLD
2,1656683135,2025-05-14T01:54:34,2025-05-14T02:04:36.193784,Senior Data Engineer,Data Engineer,GROW Inc,https://www.linkedin.com/company/grow-inc,2025-06-13T01:53:47,"[{'@type': 'Place', 'address': {'@type': 'Post...",NaN,...,"['fintech', 'blockchain', 'DLT', 'administrati...","['Sydney, NSW 2010, AU', 'Melbourne, AU']","GROW Inc develops powerful, modern technology ...",False,Mid-Senior level,True,grow-inc,2025-05-14,Melbourne,VIC
3,1656678227,2025-05-14T01:53:39,2025-05-14T02:03:03.503493,Senior Data Engineer,Data Engineer,GROW Inc,https://www.linkedin.com/company/grow-inc,2025-06-13T01:52:55,"[{'@type': 'Place', 'address': {'@type': 'Post...",NaN,...,"['fintech', 'blockchain', 'DLT', 'administrati...","['Sydney, NSW 2010, AU', 'Melbourne, AU']","GROW Inc develops powerful, modern technology ...",False,Mid-Senior level,True,grow-inc,2025-05-14,Sydney,NSW
4,1656596364,2025-05-14T01:09:42,2025-05-14T01:33:27.129563,Principal Data Engineer,Data Engineer,MAP TALENT,https://www.linkedin.com/company/maptalentgroup,2025-06-13T01:09:42,"[{'@type': 'Place', 'address': {'@type': 'Post...",NaN,...,"['Networking', 'UX / UI', 'Digital', 'Headhunt...","['2/285 Clarence St, Sydney, New South Wales 2...",MAP Talent Group are a specialised recruitment...,True,Mid-Senior level,True,maptalentgroup,2025-05-14,Melbourne,VIC


In [11]:
#Extract the employment type from the EMPLOYMENT_TYPE field

def extract_employment_type(df_daily_all):
    """
    Extracts the employment type from the given text using regex.
    """
    df_daily_all['employment_type'] = (
        df_daily_all['EMPLOYMENT_TYPE']
        .astype(str)
        .str.replace(r"[\[\]']", '', regex=True)
        .str.strip()
    )

    df_daily_all.drop(columns=['EMPLOYMENT_TYPE'], inplace=True)

    return df_daily_all['employment_type']


df_daily_all['employment_type'] = extract_employment_type(df_daily_all)
df_daily_all['employment_type'].tail()

48    FULL_TIME
49    TEMPORARY
50    FULL_TIME
51    FULL_TIME
52    FULL_TIME
Name: employment_type, dtype: object

In [12]:
#Extract the employment size
def extract_employee_size(LINKEDIN_ORG_SIZE):
    """
    Extracts the employee size from the given text using regex.
    """
    df_daily_all['org_size'] = (
        df_daily_all['LINKEDIN_ORG_SIZE']
        .astype(str)
        .str.replace(r"employees", '', regex=True)
        .str.strip()
    )

    return df_daily_all['org_size']


df_daily_all['org_size'] = extract_employee_size(df_daily_all['LINKEDIN_ORG_SIZE'])
df_daily_all['org_size'].tail()

48    10,001+
49    10,001+
50    10,001+
51      11-50
52    201-500
Name: org_size, dtype: object

In [13]:
df_daily_all.columns = df_daily_all.columns.str.upper()
df_daily_all.head(2)

,ID,DATE_POSTED,DATE_CREATED,TITLE,JOB_CATEGORY,ORGANIZATION,ORGANIZATION_URL,DATE_VALIDTHROUGH,LOCATIONS_RAW,LOCATION_TYPE,...,LINKEDIN_ORG_DESCRIPTION,LINKEDIN_ORG_RECRUITMENT_AGENCY_DERIVED,SENIORITY,DIRECTAPPLY,LINKEDIN_ORG_SLUG,JOB_DATE,CITY,STATE,EMPLOYMENT_TYPE,ORG_SIZE
0,1656925568,2025-05-14T03:20:29,2025-05-14T03:32:52.435808,Data Engineer with Databricks,Data Engineer,BURGEON IT SERVICES,https://www.linkedin.com/company/burgeon-it-se...,2025-06-13T03:20:29,"[{'@type': 'Place', 'address': {'@type': 'Post...",NaN,...,Burgeon is a provider of technical and IT staf...,True,Mid-Senior level,True,burgeon-it-services-llc,2025-05-14,Melbourne,VIC,CONTRACTOR,51-200
1,1656683181,2025-05-14T01:55:49,2025-05-14T02:04:37.175479,Data Engineer - DataStage | REDSHIFT | SQL | P...,Data Engineer,Resource Corner,https://www.linkedin.com/company/resource-corn...,2025-06-13T01:55:49,"[{'@type': 'Place', 'address': {'@type': 'Post...",NaN,...,Resource Corner is a Leading IT Consulting and...,False,Entry level,True,resource-cornerit,2025-05-14,Brisbane,QLD,FULL_TIME,201-500


In [14]:
#Only keep the relevant columns

df_daily_all = df_daily_all[['ID', 'TITLE', 'JOB_CATEGORY',
       'JOB_DATE', 'CITY', 'STATE', 'EMPLOYMENT_TYPE' ,
       'ORGANIZATION', 'ORGANIZATION_URL', 'URL',
       'SOURCE_TYPE', 'SOURCE', 'SOURCE_DOMAIN',
       'ORGANIZATION_LOGO', 'REMOTE_DERIVED', 'RECRUITER_NAME', 'RECRUITER_TITLE',
       'RECRUITER_URL', 'LINKEDIN_ORG_URL',
       'ORG_SIZE', 'LINKEDIN_ORG_INDUSTRY',
       'LINKEDIN_ORG_HEADQUARTERS',
       'LINKEDIN_ORG_TYPE', 'LINKEDIN_ORG_FOUNDEDDATE',
       'LINKEDIN_ORG_SPECIALTIES', 'LINKEDIN_ORG_LOCATIONS',
       'LINKEDIN_ORG_DESCRIPTION','LINKEDIN_ORG_RECRUITMENT_AGENCY_DERIVED',
       'SENIORITY', 'DIRECTAPPLY',
       'LINKEDIN_ORG_SLUG']]


df_daily_all.head()

,ID,TITLE,JOB_CATEGORY,JOB_DATE,CITY,STATE,EMPLOYMENT_TYPE,ORGANIZATION,ORGANIZATION_URL,URL,...,LINKEDIN_ORG_HEADQUARTERS,LINKEDIN_ORG_TYPE,LINKEDIN_ORG_FOUNDEDDATE,LINKEDIN_ORG_SPECIALTIES,LINKEDIN_ORG_LOCATIONS,LINKEDIN_ORG_DESCRIPTION,LINKEDIN_ORG_RECRUITMENT_AGENCY_DERIVED,SENIORITY,DIRECTAPPLY,LINKEDIN_ORG_SLUG
0,1656925568,Data Engineer with Databricks,Data Engineer,2025-05-14,Melbourne,VIC,CONTRACTOR,BURGEON IT SERVICES,https://www.linkedin.com/company/burgeon-it-se...,https://au.linkedin.com/jobs/view/data-enginee...,...,"Claymont, Delaware",Privately Held,2010.0,"['IT Consulting', 'Contract Staffing', 'Perman...","['619,, New York Avenue, Claymont, Delaware 19...",Burgeon is a provider of technical and IT staf...,True,Mid-Senior level,True,burgeon-it-services-llc
1,1656683181,Data Engineer - DataStage | REDSHIFT | SQL | P...,Data Engineer,2025-05-14,Brisbane,QLD,FULL_TIME,Resource Corner,https://www.linkedin.com/company/resource-corn...,https://au.linkedin.com/jobs/view/data-enginee...,...,"Norwest, New South Wales",Privately Held,2006.0,[''],"['4 Columbia Court, Norwest Business Park, Lev...",Resource Corner is a Leading IT Consulting and...,False,Entry level,True,resource-cornerit
2,1656683135,Senior Data Engineer,Data Engineer,2025-05-14,Melbourne,VIC,FULL_TIME,GROW Inc,https://www.linkedin.com/company/grow-inc,https://au.linkedin.com/jobs/view/senior-data-...,...,"Sydney, NSW",Privately Held,2017.0,"['fintech', 'blockchain', 'DLT', 'administrati...","['Sydney, NSW 2010, AU', 'Melbourne, AU']","GROW Inc develops powerful, modern technology ...",False,Mid-Senior level,True,grow-inc
3,1656678227,Senior Data Engineer,Data Engineer,2025-05-14,Sydney,NSW,FULL_TIME,GROW Inc,https://www.linkedin.com/company/grow-inc,https://au.linkedin.com/jobs/view/senior-data-...,...,"Sydney, NSW",Privately Held,2017.0,"['fintech', 'blockchain', 'DLT', 'administrati...","['Sydney, NSW 2010, AU', 'Melbourne, AU']","GROW Inc develops powerful, modern technology ...",False,Mid-Senior level,True,grow-inc
4,1656596364,Principal Data Engineer,Data Engineer,2025-05-14,Melbourne,VIC,FULL_TIME,MAP TALENT,https://www.linkedin.com/company/maptalentgroup,https://au.linkedin.com/jobs/view/principal-da...,...,"Sydney, New South Wales",Privately Held,2015.0,"['Networking', 'UX / UI', 'Digital', 'Headhunt...","['2/285 Clarence St, Sydney, New South Wales 2...",MAP Talent Group are a specialised recruitment...,True,Mid-Senior level,True,maptalentgroup


## Connect to  Snowflake database for raw data initial processing

In [15]:
#Establish a connection to Snowflake

def connect_to_snowflake():
    try:

        conn = snowflake.connector.connect(
            user=snowflake_user,
            password=snowflake_password,
            account=account,
            warehouse="SNOWFLAKE_LEARNING_WH",
            database="linkedin_db",
            schema="linkedin_raw"
        )
        print("Connection to Snowflake established successfully.")
        return conn
    except Exception as e:
        print(f"Error connecting to Snowflake: {e}")
        return None

conn = connect_to_snowflake()

Connection to Snowflake established successfully.


In [16]:
#query the raw data and narrow down to DE, DS and DA roles
#This data is filtered by relevant roles, and translated into English (but no city, state and seniority fix)
def query_existing_job_data(conn):
    query = """
        SELECT * FROM LINKEDIN_JOB_API_CLEANED_DATA
        WHERE (
            lower(TITLE) LIKE '%data engineer%'
            OR lower(TITLE) LIKE '%data scientist%'
            OR lower(TITLE) LIKE '%data analyst%'
            )
    """

    df = pd.read_sql(query, conn)
    print(df.shape)
    return df

df = query_existing_job_data(conn)
df.head()

/tmp/ipykernel_7284/824833193.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


(477, 31)


,ID,TITLE,JOB_CATEGORY,JOB_DATE,CITY,STATE,EMPLOYMENT_TYPE,ORGANIZATION,ORGANIZATION_URL,URL,...,LINKEDIN_ORG_HEADQUARTERS,LINKEDIN_ORG_TYPE,LINKEDIN_ORG_FOUNDEDDATE,LINKEDIN_ORG_SPECIALTIES,LINKEDIN_ORG_LOCATIONS,LINKEDIN_ORG_DESCRIPTION,LINKEDIN_ORG_RECRUITMENT_AGENCY_DERIVED,SENIORITY,DIRECTAPPLY,LINKEDIN_ORG_SLUG
0,1599037955,Senior Data Engineer,Data Engineer,2025-04-28,Greater Sydney Area,NA,FULL_TIME,One51 | Data & Analytics Consultancy,https://www.linkedin.com/company/one51consulting,https://au.linkedin.com/jobs/view/senior-data-...,...,"Sydney, NSW",Privately Held,2020.0,"['Business Intelligence', 'Data Warehousing', ...","['333 George Street, Level 13, Sydney, NSW 200...",Drawing on a wealth of expertise and a deep un...,False,Mid-Senior level,False,one51consulting
1,1598878880,Senior Data Engineer,Data Engineer,2025-04-28,Brisbane,QLD,CONTRACTOR,Data#3,https://www.linkedin.com/company/data3,https://au.linkedin.com/jobs/view/senior-data-...,...,"Toowong, Queensland",Public Company,1977.0,"['Cloud Solutions', 'Mobility Solutions', 'Sec...","['555 Coronation Dr, Toowong, Queensland 4066,...","Data#3 Limited (DTL), is focused on helping cu...",False,Medium-high level,False,data3
2,1598880069,Data Engineer,Data Engineer,2025-04-28,Sydney,NSW,CONTRACTOR,Whizdom,https://www.linkedin.com/company/whizdom-recru...,https://au.linkedin.com/jobs/view/data-enginee...,...,"Canberra, Australian Capital Territory",Privately Held,2006.0,"['IT Recruitment', 'Recruitment for Government...","['28-34 Thynne St, Unit 7, Canberra, Australia...","Established in 2006, Whizdom is an Australian ...",True,Middle level,True,whizdom-recruitment
3,1598878894,Senior Data Engineer and Business Intelligence...,Data Engineer,2025-04-28,Moreton Bay,QLD,FULL_TIME,University of the Sunshine Coast,https://www.linkedin.com/school/university-of-...,https://au.linkedin.com/jobs/view/senior-data-...,...,None,None,NaN,None,None,None,None,Not Applicable,False,None
4,1598880083,Senior Data Engineer and Business Intelligence...,Data Engineer,2025-04-28,Sunshine Coast,QLD,FULL_TIME,University of the Sunshine Coast,https://www.linkedin.com/school/university-of-...,https://au.linkedin.com/jobs/view/senior-data-...,...,None,None,NaN,None,None,None,None,Not Applicable,False,None


In [17]:
df.to_csv('api_cleaned_job_id_snowflake.csv', index=False)

In [18]:
list_daily = df_daily_all.columns.tolist()
list_df = df.columns.tolist()
list_daily ==  list_df

True

In [19]:
#Check the Job ID from df and only keep those new jobs based on the Job IDs
def keep_new_jobs():
    existing_job_ids = df.ID.unique().tolist()

    df_new_jobs = df_daily_all[~(df_daily_all['ID'].isin(existing_job_ids))].reset_index(drop=True)
    print(f'{df_new_jobs.shape[0]} new jobs ready to load to Snowflake')
    return df_new_jobs

df_new_jobs = keep_new_jobs()
df_new_jobs.tail()

53 new jobs ready to load to Snowflake


,ID,TITLE,JOB_CATEGORY,JOB_DATE,CITY,STATE,EMPLOYMENT_TYPE,ORGANIZATION,ORGANIZATION_URL,URL,...,LINKEDIN_ORG_HEADQUARTERS,LINKEDIN_ORG_TYPE,LINKEDIN_ORG_FOUNDEDDATE,LINKEDIN_ORG_SPECIALTIES,LINKEDIN_ORG_LOCATIONS,LINKEDIN_ORG_DESCRIPTION,LINKEDIN_ORG_RECRUITMENT_AGENCY_DERIVED,SENIORITY,DIRECTAPPLY,LINKEDIN_ORG_SLUG
48,1656412981,Senior Data Analyst – People Analytics,Data Analyst,2025-05-14,Southbank,VIC,FULL_TIME,Crown Resorts,https://www.linkedin.com/company/crown-resorts,https://au.linkedin.com/jobs/view/senior-data-...,...,"Melbourne, Victoria",Privately Held,1994.0,"['Hotels', 'Food & Beverage', 'Capital Project...","['8 Whiteman Street Southbank, Melbourne, Vict...",Crown Resorts is one of Australia’s largest en...,False,Mid-Senior level,False,crown-resorts
49,1655203763,Quality Assurance and Data Analyst,Data Analyst,2025-05-13,St Leonards,NSW,TEMPORARY,NSW Health,https://www.linkedin.com/company/nsw-health,https://au.linkedin.com/jobs/view/quality-assu...,...,"St. Leonards, New South Wales",Government Agency,NaN,[''],"['1 Reserve Road, St. Leonards, New South Wale...","​​​​​​​With more than 170,000 staff and 228 ho...",False,Mid-Senior level,False,nsw-health
50,1654707679,Data Analyst,Data Analyst,2025-05-13,Sydney,NSW,FULL_TIME,Stryker,https://www.linkedin.com/company/stryker,https://au.linkedin.com/jobs/view/data-analyst...,...,"Kalamazoo, MI",Public Company,1941.0,"['medical Instruments & Supplies', 'medical de...","['2825 Airview Boulevard, Kalamazoo, MI 49002,...",Stryker is a global leader in medical technolo...,False,Associate,False,stryker
51,1654638281,Senior Data Analyst,Data Analyst,2025-05-13,Wales,None,FULL_TIME,Archer - The IT Recruitment Consultancy,https://www.linkedin.com/company/archer,https://uk.linkedin.com/jobs/view/senior-data-...,...,Dublin,Privately Held,2007.0,"['Java & .NET developer jobs', 'IT Business An...","['2 Westland Square, Pearse Street, Dublin, 2,...",Archer works with Ireland's top IT employers t...,True,Mid-Senior level,True,archer
52,1653726329,"Data Analyst (Data Governance + SQL, + AML/KYC)",Data Analyst,2025-05-13,Melbourne,VIC,FULL_TIME,CareCone Group,https://www.linkedin.com/company/carecone,https://au.linkedin.com/jobs/view/data-analyst...,...,"Canberra, Australian Capital Territory",Privately Held,2015.0,"['Recruitment', 'Payroll', 'Managed Services',...","['26 Longmore Cres, Wanniassa, Canberra, Austr...",CareCone Consulting was founded as a result of...,True,Orta-Üst Düzey Yönetici,True,carecone


In [20]:
#Translate new job's business name, city, state and seniority to English

translator = Translator()

# def translate_text(text, target_language='en'):
#     try:
#         if not text or pd.isna(text):
#             return 'NA'
#         translated_text = translator.translate(str(text), dest=target_language)
#         return translated_text.text
#     except Exception as e:
#         print(f"Error translating text: {e} (Text: {text})")
#         return text

# for col in ['CITY', 'STATE', 'ORGANIZATION', 'SENIORITY']:
#     unique_values = df_new_jobs[col].dropna().unique()
#     translation_map = {val: translate_text(val, target_language='en') for val in unique_values}
#     df_new_jobs[col] = df_new_jobs[col].map(translation_map).fillna('NA')


async def translate_text(translator, text, target_language='en'):
    try:
        if not text or pd.isna(text):
            return 'NA'
        result = await translator.translate(str(text), dest=target_language)
        return result.text
    except Exception as e:
        print(f"Error translating text: {e} (Text: {text})")
        return text

async def translate_column(df, col, target_language='en'):
    translator = Translator()
    unique_values = df[col].dropna().unique()
    # 并发批量异步翻译（速度更快）
    tasks = {val: translate_text(translator, val, target_language) for val in unique_values}
    translation_map = {}
    for val, task in tasks.items():
        translation_map[val] = await task
    df[col] = df[col].map(translation_map).fillna('NA')

async def main():
    # 假设 df_new_jobs 已准备好
    for col in ['CITY', 'STATE', 'ORGANIZATION', 'SENIORITY']:
        await translate_column(df_new_jobs, col, 'en')

# 运行主协程
await main()

In [21]:
df_new_jobs['CITY'].unique()

array(['Melbourne', 'Brisbane', 'Sydney', 'Canberra',
       'Greater Sydney Area', 'North Parramatta', 'Woden', 'Perth',
       'Other Side Of The Moon', 'Newcastle', 'Hobart', 'Dandenong',
       'Box Hill', 'Adelaide', 'Moonee Ponds', 'Parramatta',
       'New South Wales', 'Southbank', 'St Leonards', 'Wales'],
      dtype=object)

In [22]:
df_new_jobs['STATE'].unique()

array(['VIC', 'QLD', 'NSW', 'NA', 'Of', 'That', 'on', 'Australia'],
      dtype=object)

In [23]:

def load_to_snowflake(df_new_jobs):
    # Create a Snowflake connection engine
    engine = create_engine(
        'snowflake://{user}:{password}@{account}/{database}/{schema}?warehouse={warehouse}'.format(
            user=snowflake_user,
            password=snowflake_password,
            account=account,
            warehouse="SNOWFLAKE_LEARNING_WH",
            database="linkedin_db",
            schema="linkedin_raw"
        )
    )

    table_name = "linkedin_job_api_cleaned_data"

    df_new_jobs.to_sql(
        name=table_name,
        con=engine,
        if_exists='append', #append data
        index=False
    )

    print(f"Data loaded to Snowflake table {table_name} successfully.")


load_to_snowflake(df_new_jobs)

Data loaded to Snowflake table linkedin_job_api_cleaned_data successfully.
